In [98]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings

In [99]:
path = kagglehub.dataset_download("jansenccruz/roblox-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/lily/.cache/kagglehub/datasets/jansenccruz/roblox-dataset/versions/1


In [100]:
roblox = pd.read_csv('/home/lily/github/Roblox_ML_Modeling/data/raw/roblox_games_data.csv')
roblox.head(3)

,Title,Creator,AgeRecommendation,Active,Favorites,Visits,VoiceChat,Camera,Created,Updated,ServerSize,Genre,Likes,Dislikes,GameLink,DateFetched
0,Catalog Avatar Creator,@ItsMuneeeb,All Ages,19469,3438844,2.8B+,Supported,Supported,7/4/2021,11/5/2024,24,Shopping,1M+,124K+,https://www.roblox.com/games/7041939546/Catalo...,11/5/2024 20:24
1,[HALLOWEEN 🎃] The Mimic,CTStudio,Ages 13+,2709,2911356,988.3M+,Supported,Not Supported,1/15/2021,11/5/2024,45,Survival,982K+,107K+,https://www.roblox.com/games/6243699076/HALLOW...,11/5/2024 20:25
2,[RANKED + MORE] Frisbee Frenzy,johnny cage fan group,All Ages,148,21024,1.5M+,Supported,Not Supported,10/26/2023,11/5/2024,8,NaN,5265,1011,https://www.roblox.com/games/15170454486/RANKE...,11/5/2024 20:25


In [101]:
roblox.dropna(subset=['Genre'], inplace=True)


In [102]:
def convert_to_numeric(value):
    if isinstance(value, str):
        value = value.lower().replace(',', '')
        if 'b+' in value:
            return float(value.replace('b+', '')) * 1_000_000_000
        elif 'm+' in value:
            return float(value.replace('m+', '')) * 1_000_000
        elif 'k+' in value:
            return float(value.replace('k+', '')) * 1_000
    try:
        return float(value)
    except (ValueError, TypeError):
        return np.nan
    
roblox['Visits'] = roblox['Visits'].apply(convert_to_numeric)
roblox['Likes'] = roblox['Likes'].apply(convert_to_numeric)
roblox['Dislikes'] = roblox['Dislikes'].apply(convert_to_numeric)

In [103]:
roblox['like_ratio'] = roblox['Likes'] / (roblox['Likes'] + roblox['Dislikes'])
roblox['like_ratio'] = roblox['like_ratio'].fillna(0)

roblox['player_to_visit_ratio'] = roblox['Active'] / roblox['Visits']

In [108]:
roblox_encoded = pd.get_dummies(
    roblox,
    columns=['Genre', 'Camera', 'VoiceChat'],  # all categorical columns
    prefix=['genre', 'camera', 'vc'],         # prefixes for each
    dtype=int
)

roblox_encoded.columns = [col.lower().replace(' ', '_') for col in roblox_encoded.columns]
roblox_encoded.columns

Index(['title', 'creator', 'agerecommendation', 'active', 'favorites',
       'visits', 'created', 'updated', 'serversize', 'likes', 'dislikes',
       'gamelink', 'datefetched', 'like_ratio', 'player_to_visit_ratio',
       'genre_action', 'genre_adventure', 'genre_education',
       'genre_entertainment', 'genre_obby_&_platformer',
       'genre_party_&_casual', 'genre_puzzle', 'genre_rpg',
       'genre_roleplay_&_avatar_sim', 'genre_shooter', 'genre_shopping',
       'genre_simulation', 'genre_social', 'genre_sports_&_racing',
       'genre_strategy', 'genre_survival', 'camera_not_supported',
       'camera_supported', 'vc_not_supported', 'vc_supported'],
      dtype='object')

In [109]:
roblox_encoded.to_csv('/home/lily/github/Roblox_ML_Modeling/data/processed/roblox_dataset_cleaned.csv', index=False)
